In [34]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree, datasets
from sklearn.model_selection import train_test_split
import sklearn
import pandas as pd

In [35]:
df = pd.read_csv("filtered_labelled_features.csv")
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
df.head()

,sequence,sid,pos_acts_toeachother,neg_acts_toeachother,pos_acts_together,neg_acts_together,pos_char1_acts,neg_char1_acts,pos_char2_acts,neg_char2_acts,...,neg,filename,unique_ID,29:posFramesFired,30:negFramesFired,31:otherFramesFired,32:posFramewrtCharFired,33:negFramewrtCharFired,34:otherFrameswrtCharFired,manualLabel
0,1,s0,0,0,0,0,0,0,0,1,...,1,aeschylus.libation.0.3.sent,aeschylus.libation.0.3.sents0,2,2,1,1,2,1,p
1,1,s9,0,0,0,0,0,0,0,0,...,0,aeschylus.libation.0.3.sent,aeschylus.libation.0.3.sents9,1,1,0,0,0,0,p
2,1,s10,0,0,0,0,0,0,1,0,...,0,aeschylus.libation.0.3.sent,aeschylus.libation.0.3.sents10,2,1,0,0,0,0,p
3,1,s12,0,0,0,0,0,0,0,0,...,0,aeschylus.libation.0.3.sent,aeschylus.libation.0.3.sents12,0,1,0,0,0,0,p
4,1,s13,0,0,0,0,0,0,0,0,...,2,aeschylus.libation.0.3.sent,aeschylus.libation.0.3.sents13,0,0,0,0,0,0,p


In [36]:
# df[df['manualLabel'] == 'p']['label'] = 1
# df[df['manualLabel'] == 'n']['label'] = 0

# df.shape
# df.head()

In [37]:
Y = df['manualLabel']
X = df.drop(['manualLabel', 'filename', 'unique_ID', 'sequence', 'sid'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = .20, random_state = 1234)

In [38]:
# training decision tree 
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

# make predictions on test set 
preds = clf.predict(X_test)

In [39]:
correct = 0 
total = 0 

for pred, target in zip(preds, y_test):
    if pred == target:
        correct += 1    
    total += 1
    
accuracy = correct/total 
print(accuracy)

0.743801652892562


In [43]:
from sklearn.metrics import average_precision_score

CF = sklearn.metrics.confusion_matrix(y_test.astype(str), preds)
print(CF)

[[ 7 16]
 [15 83]]


In [45]:
labs = ['n', 'p']

FP = {}
for r, label in enumerate(labs):
    fp = 0
    for c in range(2):
        if r != c: 
            fp += CF[r][c]
        FP[label] = fp           
print(FP)


FN = {}
for c, label in enumerate(labs):
    fn = 0 
    for r in range(2):
        if c != r: 
            fn += CF[r][c]
        FN[label] = fn
print(FN)

TP = {}
for r, label in enumerate(labs):
    for c in range(2):
        if r == c:
            TP[label] = CF[r][c]

print(TP)


{'n': 16, 'p': 15}
{'n': 15, 'p': 16}
{'n': 7, 'p': 83}


In [46]:
# calculating precision and recall 

# precision = TP / TP + FP 

# recal = TP / TP + FN 

results = {'n': {}, 'p': {}}

for lab in labs:
    results[lab]['precision'] = TP[lab] / (TP[lab] + FP[lab])
    results[lab]['recall'] = TP[lab] / (TP[lab] + FN[lab])
    results[lab]['F1'] = 2 * ((results[lab]['precision'] * results[lab]['recall']) / (results[lab]['precision'] + results[lab]['recall']))

    

print(results)
    
    

{'n': {'precision': 0.30434782608695654, 'recall': 0.3181818181818182, 'F1': 0.31111111111111117}, 'p': {'precision': 0.8469387755102041, 'recall': 0.8383838383838383, 'F1': 0.8426395939086294}}


In [47]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy_score(y_test.astype('str'), preds.astype('str'))

0.743801652892562

In [48]:
precision_score(y_test.astype('str'), preds.astype('str'), average=None)

array([0.31818182, 0.83838384])

In [49]:
recall_score(y_test.astype('str'), preds.astype('str'), average=None)

array([0.30434783, 0.84693878])

In [50]:
f1_score(y_test.astype('str'), preds.astype('str'), average=None)

array([0.31111111, 0.84263959])